# Experiment B

Objective: Train a 1D-CNN Image Encoder on the SPH dataset.

Split the data into train, validation, test subsets.

Example code for finding f1_thresholds

```
def find_threshold_f1(trues, logits, eps=1e-9):
    precision, recall, thresholds = precision_recall_curve(trues, logits)
    f1_scores = 2 * precision * recall / (precision + recall + eps)
    threshold = float(thresholds[np.argmax(f1_scores)])  
    return threshold
```

trues = true labels (binarized)
logits = row outputs of the model (sigmoid output/probabilties)

For each label, there will be individual thresholds.

Filter out class samples where value counts < 100.

In [ ]:
import sys
import h5py
from glob import glob
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score, average_precision_score, f1_score, accuracy_score, precision_score, recall_score, f1_score

from albumentations.pytorch import ToTensorV2
from transformers import AutoTokenizer, AutoModel
import albumentations as A

from sklearn.model_selection import train_test_split
from tqdm import tqdm
import cv2
import random
import tarfile

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# Use a fixed seed value
set_seed(42)

## Notebook Setup

In [ ]:
class CONFIG:
    debug = False
    batch_size = 256
    num_workers = 8
    head_lr = 0.001
    image_encoder_lr = 0.001
    patience = 5
    factor = 0.8
    epochs = 20
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Image Model
    model_name = 'resnet18'
    image_embedding_size = 512

    # Text Model
    text_encoder_model = 'emilyalsentzer/Bio_ClinicalBERT'
    text_tokenizer = 'emilyalsentzer/Bio_ClinicalBERT'
    text_embedding_size = 768
    max_length = 200

    pretrained = True # for both image encoder and text encoder
    trainable = True # for both image encoder and text encoder
    temperature = 10.0
    optimizer = torch.optim.Adam

    # image size
    size = 224

    # for projection head; used for both image and text encoder
    num_projection_layers = 1
    projection_dim = 128
    dropout = 0.0
    ecg_sr = 128

In [ ]:
_ACTIVATION_DICT = {'relu': nn.ReLU,
                    'tanh': nn.Tanh,
                    'none': nn.Identity,
                    'leaky_relu': lambda: nn.LeakyReLU(negative_slope=0.2)}


class Conv1dBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size,
                 act='relu', bn=True, dropout=None,
                 maxpool=None, padding=None, stride=1):

        super().__init__()

        if padding is None or padding == 'same':
            padding = kernel_size // 2

        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, padding=padding, bias=not bn, stride=stride)
        self.bn = nn.BatchNorm1d(out_channels) if bn else None
        self.act = _ACTIVATION_DICT[act]()
        self.dropout = None if dropout is None else nn.Dropout(dropout)
        self.maxpool = None if maxpool is None else nn.MaxPool1d(maxpool)

    def forward(self, x):
        x = self.conv(x)

        if self.bn is not None:
            x = self.bn(x)

        x = self.act(x)

        if self.dropout is not None:
            x = self.dropout(x)

        if self.maxpool is not None:
            x = self.maxpool(x)

        return x


class LinearBlock(nn.Module):
    def __init__(self, in_channels, out_channels, act='relu', bn=True, dropout=None):

        super().__init__()

        self.linear = nn.Linear(in_channels, out_channels, bias=not bn)
        self.bn = nn.BatchNorm1d(out_channels) if bn else None
        self.act = _ACTIVATION_DICT[act]()
        self.dropout = None if dropout is None else nn.Dropout(dropout)

    def forward(self, x):
        x = self.linear(x)

        if self.bn is not None:
            x = self.bn(x)

        x = self.act(x)

        if self.dropout is not None:
            x = self.dropout(x)
        return x


class ConvEncoder(nn.Module):
    def __init__(self, in_channels, channels, kernels, bn=True, dropout=None, maxpool=2, padding=0, stride=None):
        super().__init__()



        num_layers = len(channels)
        if stride is None:
            stride = [1] * num_layers

        self.in_layer = Conv1dBlock(in_channels, channels[0], kernels[0], bn=bn, dropout=dropout, maxpool=maxpool, padding=padding, stride=stride[0])

        conv_layers = list()
        for i in range(1, num_layers):
            conv_layers.append(Conv1dBlock(channels[i - 1], channels[i], kernels[i], bn=bn, dropout=dropout, maxpool=maxpool, padding=padding, stride=stride[i]))
        self.conv_layers = nn.ModuleList(conv_layers)

    def forward(self, x):
        x = self.in_layer(x)
        for layer in self.conv_layers:
            x = layer(x)
        return x


class ECGEncoder(nn.Module):
    def __init__(self,
                 window=1280,
                 in_channels=12,
                 channels=(32, 32, 64, 64, 128, 128, 256, 256),
                 kernels=(7, 7, 5, 5, 3, 3, 3, 3),
                 linear=512,
                 output=512):

        super().__init__()

        self.conv_encoder = ConvEncoder(in_channels, channels,  kernels, bn=True)

        with torch.no_grad():
            inpt = torch.zeros((1, in_channels, window), dtype=torch.float32)
            outpt = self.conv_encoder(inpt)
            output_window = outpt.shape[2]

        self.flatten = nn.Flatten()
        self.conv_to_linear = nn.Linear(output_window * channels[-1], linear)
        self.act = nn.ReLU()
        self.out_layer = nn.Linear(linear, output)

    def forward(self, x):
        x = self.conv_encoder(x)
        x = self.flatten(x)
        x = self.conv_to_linear(x)
        x = self.act(x)
        x = self.out_layer(x)
        return x

## Loading Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
class ImageEncoder(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.config = CONFIG
        self.encoder = ECGEncoder(output=CONFIG.image_embedding_size)
        # Add some non-linear activation here like RELU because ECG encoder already returns linear layer so this will help.
        self.fc = nn.Linear(CONFIG.image_embedding_size, 47)  # Added this line

    def forward(self, x):
        x = self.encoder(x)
        x = self.fc(x)  # Added this line
        return x

## Model Training Setup

### Create X_train, y_train, X_test, y_test variables